In [ ]:
# 初期化

import os
import cx_Oracle as cx

import config

os.environ['TNS_ADMIN'] = config.tns_admin_path
user = config.user
password = config.password
dsn = config.dsn



In [ ]:
# 直近30秒のキーワードランキング

with cx.connect(user=user, password=password, dsn=dsn) as conn:
    cur = conn.cursor()
    cur.execute("""
        select WINDOW_END + 9/24 as JST, KEYWORD, APPEARANCES from demo.TWEET_KEYWORDS
          where WINDOW_END = (select max(WINDOW_END) from demo.TWEET_KEYWORDS)
            order by APPEARANCES desc
    """)
    res = cur.fetchall()
    #res = cur.fetchmany(10)
    for r in res:
        print (r)


In [ ]:
# 直近30秒のワードクラウド

words = []
end_time = None

with cx.connect(user=user, password=password, dsn=dsn) as conn:
    cur = conn.cursor()
    cur.execute("""
        select WINDOW_END + 9/24 as JST, KEYWORD, APPEARANCES from demo.TWEET_KEYWORDS
          where WINDOW_END = (select max(WINDOW_END) from demo.TWEET_KEYWORDS)
            order by APPEARANCES desc
    """)
    res = cur.fetchall()
    for (end, word, count) in res:
        end_time = end
        for i in range(count):
            words.append(word)

#print('|'.join(words))

### create word cloud
from wordcloud import WordCloud
 
#fpath = "/usr/share/fonts/ipa-pgothic/ipagp.ttf"
fpath = "/usr/share/fonts/ipa-gothic/ipag.ttf"
wordcloud = WordCloud(
    background_color="white",
    font_path=fpath,
    collocations=False,
    width=800, height=600,
    min_font_size=15)
wordcloud.generate(' '.join(words))
wordcloud.to_file('./wordcloud.png')

### show wordcloud image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#print(f'Datetime: {end_time.isoformat()}')
plt.figure(figsize=(8,6))
ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
img = mpimg.imread('./wordcloud.png')
plt.title(f"{end_time.isoformat()} JST", loc='right', pad=10)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.imshow(img)


In [ ]:
# All Time ランキング (Top 100)

with cx.connect(user=user, password=password, dsn=dsn) as conn:
    cur = conn.cursor()

    cur.execute("""
      select min(WINDOW_START) + 9/24, max(WINDOW_END) + 9/24 from demo.TWEET_KEYWORDS
    """)
    r = cur.fetchone()
    print(f'From {r[0].isoformat()} To {r[1].isoformat()}')

    cur.execute("""
      with HISTORY as (
        select KEYWORD, sum(APPEARANCES) as TOTAL from demo.TWEET_KEYWORDS
          group by KEYWORD
            order by TOTAL desc
      )
      select * from HISTORY where ROWNUM <= 100
    """)
    res = cur.fetchall()
    for r in res:
        print (r)